In [8]:
# https://nbviewer.jupyter.org/github/PuneetGrov3r/MediumPosts/blob/master/Tackle/BigData-IncrementalLearningAndDask.ipynb#Method-2:-Using-Dask:
# https://towardsdatascience.com/how-to-learn-from-bigdata-files-on-low-memory-incremental-learning-d377282d38ff#2d3d
# https://towardsdatascience.com/speeding-up-your-algorithms-part-4-dask-7c6ed79994ef
# https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html

# Import database
import numpy as np
import pandas as pd

import os
import sys
import time
import timeit
sys.path.insert(0, '../../../../Utilities/')

from plotting import newfig, savefig
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

import operator
import itertools

from sklearn import metrics
from sklearn.metrics import *

from sklearn import preprocessing

from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.inspection import permutation_importance

from joblib import dump, load
import pickle

from sklearn.tree import DecisionTreeRegressor

from sklearn.multioutput import MultiOutputRegressor

n_jobs = 1
trial  = 1

import bz2
from shutil import copyfileobj

import dask

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import dask.delayed as delay
from dask.distributed import Client
from dask_ml.model_selection import train_test_split, GridSearchCV
client = Client(processes=False, threads_per_worker=2, n_workers=2, memory_limit='2GB')
client

/home/lk/.local/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43579 instead
  http_address["port"], self.http_server.port


Client Scheduler: inproc://10.0.0.119/26422/12 Dashboard: http://10.0.0.119:43579/status,Cluster Workers: 2 Cores: 4 Memory: 4.00 GB


In [22]:
df = dd.read_fwf("../../../Data/TCs_air5_MD.txt", blocksize=10e6)
data = pd.DataFrame(df).to_numpy()
darr = da.from_array(data, chunks=(1000,1000))
#df.npartitions
darr.npartitions
#df.visualize(size="7,5!")

#result = darr.compute()

8

In [15]:
df.head()

,T,P,x_N2,x_O2,x_NO,x_N,x_O,Eta,Zeta,Lam,...,[110],[111] [112] [113] [114],[115],[116],[117],[118],[119],[120],[121],Unnamed: 129
0,500.0,5000.0,0.255530,0.065771,0.286183,0.261033,0.072110,0.041204,0.000029,0.000019,...,-6.097647e-47,-2.114684e-48 -7.911150e-50 -3.191995e-51 -...,-3.293641e-55,-1.794685e-56,-1.053698e-57,-6.664651e-59,-4.540357e-60,-3.330974e-61,-0.000078,-0.000083
1,500.0,5000.0,0.164873,0.018555,0.129873,0.007793,0.678906,0.044471,0.000030,0.000019,...,1.927782e-46,6.685612e-48 2.501125e-49 1.009155e-50 4...,1.041291e-54,5.673928e-56,3.331285e-57,2.107042e-58,1.435442e-59,1.053094e-60,-0.000063,-0.000067
2,500.0,5000.0,0.072908,0.311170,0.251743,0.291534,0.072646,0.041844,0.000029,0.000019,...,-1.837734e-47,-6.373319e-49 -2.384295e-50 -9.620166e-52 -...,-9.926512e-56,-5.408892e-57,-3.175677e-58,-2.008620e-59,-1.368391e-60,-1.003902e-61,-0.000077,-0.000082
3,500.0,5000.0,0.065731,0.103583,0.057109,0.231777,0.541800,0.044239,0.000030,0.000019,...,-1.202527e-47,-4.170405e-49 -1.560171e-50 -6.294992e-52 -...,-6.495449e-56,-3.539328e-57,-2.078016e-58,-1.314347e-59,-8.954120e-61,-6.569073e-62,-0.000059,-0.000062
4,500.0,5000.0,0.280106,0.103732,0.073565,0.480825,0.061772,0.041883,0.000029,0.000019,...,8.327097e-47,2.887864e-48 1.080366e-49 4.359068e-51 1...,4.497879e-55,2.450866e-56,1.438956e-57,9.101413e-59,6.200423e-60,4.548861e-61,-0.000068,-0.000071


In [78]:
# Convert string Series to dictionary Series
#df['Dict'] = df['Dict'].apply(lambda x: json.loads(x.replace("'", '"')), meta=('Dict', 'f8'))
#
#dict_col_keys = {
#    'Dict': ['MedInc', 'HouseAge']
#}
#
#for dic_col in dict_col_keys:
#  for key in dict_col_keys[dic_col]:
#    df[f'{dic_col}.{key}'] = df[dic_col].to_bag().pluck(key).to_dataframe().iloc[:,0]
#
#df.columns
#df.x_N2

Dask Series Structure:
npartitions=10
    float64
        ...
     ...   
        ...
        ...
Name: x_N2, dtype: float64
Dask Name: getitem, 20 tasks

In [31]:
# Necessary for converting dataframe to array. Takes specified length from each block.
lengths = []
for part in df.partitions:
  l = part.shape[0].compute()
  lengths.append(l)
  print(l, part.shape[1])

4972 130
4973 130
4972 130
4973 130
4973 130
4972 130
4973 130
4973 130
4972 130
2047 130


In [68]:
#data = pd.DataFrame(df)#.to_numpy()
#print(data)
X, y = df.to_dask_array(lengths=lengths) , df.to_dask_array(lengths=lengths)
print(X)
#X
print(y)
y
#df

dask.array<values, shape=(46800, 130), dtype=object, chunksize=(4973, 130), chunktype=numpy.ndarray>
dask.array<values, shape=(46800, 130), dtype=object, chunksize=(4973, 130), chunktype=numpy.ndarray>


dask.array<values, shape=(46800, 130), dtype=object, chunksize=(4973, 130), chunktype=numpy.ndarray>

In [89]:
from dask_ml.preprocessing import StandardScaler

sc_x = StandardScaler()
sc_y = StandardScaler()

Xo = dask.array.zeros((X.shape[0],1), chunks=(200000,1))

#for i, col_ in enumerate(df.columns):
#    y = sc_y.fit_transform(y.reshape(-1, 1)).reshape(1, -1)[0]
#    temp = sc_x.fit_transform(X[:,i].reshape(-1, 1))
#    Xo = dask.array.concatenate([Xo, temp], axis=1)

In [44]:
from dask_ml.preprocessing import StandardScaler

sc_x = StandardScaler()
sc_y = StandardScaler()

#df["column"] = rsc.fit_transform(df["column"])

sc_x.fit(x_train)
x_train = sc_x.fit_transform(x_train)
x_test  = sc_x.fit_transform(x_test)

sc_y.fit(y_train)
y_train = sc_y.transform(y_train)
y_test  = sc_y.transform(y_test)

NameError: name 'x_train' is not defined

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, test_size=0.25, random_state=69)

In [ ]:
#gs = GridSearchCV(estimator, param_grid, cv=10)
#gs.fit(xtr, ytr)

In [22]:
X, y = df.drop(['Eta'], axis=1).to_dask_array(lengths=lengths) , df['Eta'].to_dask_array(lengths=lengths)
print(X)
print(y)
Xo = dask.array.zeros((X.shape[0],1), chunks=(200000,1))

dask.array<values, shape=(46800, 129), dtype=object, chunksize=(4973, 129), chunktype=numpy.ndarray>
dask.array<values, shape=(46800,), dtype=float64, chunksize=(4973,), chunktype=numpy.ndarray>


In [23]:
from dask_ml.preprocessing import RobustScaler
for i, col_ in enumerate(df.columns):
  if col_ == "Target":
    rsc = RobustScaler()
    y = rsc.fit_transform(y.reshape(-1, 1)).reshape(1, -1)[0]
  else:
    rsc = RobustScaler()
    temp = rsc.fit_transform(X[:,i].reshape(-1, 1))
    Xo = dask.array.concatenate([Xo, temp], axis=1)

Xo = Xo[:, 1:]

#Xo[-5:].compute()

TypeError: operand type(s) all returned NotImplemented from __array_ufunc__(<ufunc 'subtract'>, '__call__', dask.array<reshape, shape=(46800, 1), dtype=object, chunksize=(4973, 1), chunktype=numpy.ndarray>, array([10000.])): 'Array', 'ndarray'

In [2]:
#df = pd.read_fwf("../../../Data/TCs_air5.txt")
#print(df)
#df = pd.read_fwf("../../../Data/TCs_air5.txt").to_numpy()
#print(df)
#print("df done!")

# https://stackoverflow.com/questions/6884903/how-to-pickle-several-txt-files-into-one-pickle/6897610
#file1=open('../../../Data/TCs_air5_MD.txt','r')
#obj=[file1.read()]
#pickle.dump(obj,open('result.i2','w'),2)

#df = pd.read_fwf("../../../Data/TCs_air5_MD.txt").to_numpy()
#print(df)
#print("df done!")

# https://stackoverflow.com/questions/9518705/big-file-compression-with-python
#with open('../../../Data/TCs_air5_MD.txt', 'rb') as input:
#with open('../../../Data/TCs_air5_MD_full.txt', 'rb') as input:
#    with bz2.BZ2File('../../../Data/TCs_air5_MD_full.txt.bz2', 'wb', compresslevel=9) as output:
#        copyfileobj(input, output)

# https://stackoverflow.com/questions/20428355/appending-column-to-frame-of-hdf-file-in-pandas/20428786#20428786
#store = pd.HDFStore('../../../Data/TCs_air5.h5',mode='w')
#store = pd.HDFStore('../../../Data/TCs_air5_MD.h5',mode='w')
#for chunk in pd.read_csv('../../../Data/TCs_air5.txt',chunksize=50000):
#for chunk in pd.read_fwf('../../../Data/TCs_air5.txt',chunksize=50000):
#for chunk in pd.read_fwf('../../../Data/TCs_air5_MD.txt',chunksize=100):
#         store.append('df', chunk)
#store.close()

# https://medium.com/towards-artificial-intelligence/efficient-pandas-using-chunksize-for-large-data-sets-c66bf3037f93
#data = pd.read_fwf('../../../Data/TCs_air5.txt')
#df = pd.read_fwf('../../../Data/TCs_air5.txt',chunksize=100)
#data = pd.read_fwf('../../../Data/TCs_air5_MD.txt',chunksize=100)
#data = pd.read_fwf('../../../Data/TCs_air5_MD.txt') # too big!
#df = pd.DataFrame(data)
#print(df)
#print("df done!")

# https://stackoverflow.com/questions/32358443/converting-a-dataset-into-an-hdf5-dataset
#frame = pd.read_fwf('../../../Data/TCs_air5.txt')
#type(frame)
#pd.core.frame.DataFrame
#hdf5store = pd.HDFStore('mydata.h5')
#hdf5store['frame'] = frame
##hdf5store
#list(hdf5store.items())
#hdf5store2 = pd.HDFStore('mydata.h5')
#list(hdf5store2.items())
#framecopy = hdf5store2['frame']
##framecopy
#framecopy = frame
#hdf5store2.close()

## https://stackoverflow.com/questions/14262433/large-data-work-flows-using-pandas
## create a store
#store = pd.HDFStore('mystore.h5')
#
## this is the key to your storage:
##    this maps your fields to a specific group, and defines
##    what you want to have as data_columns.
##    you might want to create a nice class wrapping this
##    (as you will want to have this map and its inversion)
#group_map = dict(
#    A = dict(fields = ['field_1','field_2'], dc = ['field_1','field_5']),
#    B = dict(fields = ['field_10'], dc = ['field_10']),
#    REPORTING_ONLY = dict(fields = ['field_1000','field_1001'], dc = []),
#)
#
#group_map_inverted = dict()
#for g, v in group_map.items():
#    group_map_inverted.update(dict([ (f,g) for f in v['fields'] ]))
#
#for f in files:
#   # read in the file, additional options may be necessary here
#   # the chunksize is not strictly necessary, you may be able to slurp each
#   # file into memory in which case just eliminate this part of the loop
#   # (you can also change chunksize if necessary)
#   for chunk in pd.read_table(f, chunksize=1000):
#       # we are going to append to each table by group
#       # we are not going to create indexes at this time
#       # but we *ARE* going to create (some) data_columns
#
#       # figure out the field groupings
#       for g, v in group_map.items():
#             # create the frame for this group
#             frame = chunk.reindex(columns = v['fields'], copy = False)
#
#             # append it
#             store.append(g, frame, index=False, data_columns = v['dc'])
#
#frame = store.select(group_that_I_want)
## you can optionally specify:
## columns = a list of the columns IN THAT GROUP (if you wanted to
##     select only say 3 out of the 20 columns in this sub-table)
## and a where clause if you want a subset of the rows
#
## do calculations on this frame
#new_frame = cool_function_on_frame(frame)
#
## to 'add columns', create a new group (you probably want to
## limit the columns in this new_group to be only NEW ones
## (e.g. so you don't overlap from the other tables)
## add this info to the group_map
#store.append(new_group, new_frame.reindex(columns = new_columns_created, copy = False), data_columns = new_columns_created)

# https://stackoverflow.com/questions/16149803/working-with-big-data-in-python-and-numpy-not-enough-ram-how-to-save-partial-r?lq=1
# https://stackoverflow.com/questions/23872942/sklearn-and-large-datasets
#a = np.memmap('test.mymemmap', dtype='float32', mode='w+', shape=(200000,1000))
## here you will see a 762MB file created in your working directory
#del a
#b = np.memmap('test.mymemmap', dtype='float32', mode='r+', shape=(200000,1000))
#b = np.memmap('test.mymemmap', dtype='float32', mode='r+', shape=(2,1000))
#b[1,5] = 123456.
#print(a[1,5])
##123456.0
#b = numpy.memmap('test.mymemmap', dtype='float32', mode='r+', shape=(2,1000), offset=150000*1000*32/8)
#b[1,2] = 999999.
#print(a[150001,2])
##999999.0

# https://towardsdatascience.com/how-to-learn-from-bigdata-files-on-low-memory-incremental-learning-d377282d38ff
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and
    modify the data type to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage of dataframe is {:.2f}' 'MB').format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            #df[col] = df[col].astype('category')
            end_mem = df.memory_usage().sum() / 1024**2
    print(('Memory usage after optimization is: {:.2f}' 'MB').format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

#reduce_mem_usage(df)
#print(df)

#incremental_dataframe = pd.read_fwf("../../../Data/TCs_air5.txt", chunksize=100) # Number of lines to read.
#print(incremental_dataframe)
#idf = pd.DataFrame(incremental_dataframe).to_numpy()
#print(idf)
#
#for df in incremental_dataframe:
#    print(0)

# http://pandas-docs.github.io/pandas-docs-travis/user_guide/io.html#io-perf
# https://stackoverflow.com/questions/16628329/hdf5-concurrency-compression-i-o-performance
# https://stackoverflow.com/questions/38515433/pickle-dump-pandas-dataframe
def test_sql_write(df):
    if os.path.exists('test.sql'):
        os.remove('test.sql')
    sql_db = sqlite3.connect('test.sql')
    df.to_sql(name='../../../Data/test_table', con=sql_db)
    sql_db.close()

def test_sql_read():
    sql_db = sqlite3.connect('../../../Data/test.sql')
    pd.read_sql_query("select * from test_table", sql_db)
    sql_db.close()

def test_hdf_fixed_write(df):
    df.to_hdf('../../../Data/test_fixed.hdf', 'test', mode='w')

def test_hdf_fixed_read():
    pd.read_hdf('../../../Data/test_fixed.hdf', 'test')

def test_hdf_fixed_write_compress(df):
    df.to_hdf('../../../Data/test_fixed_compress.hdf', 'test', mode='w', complib='blosc')

def test_hdf_fixed_read_compress():
    pd.read_hdf('../../../Data/test_fixed_compress.hdf', 'test')

def test_hdf_table_write(df):
    df.to_hdf('../../../Data/test_table.hdf', 'test', mode='w', format='table')

def test_hdf_table_read():
    pd.read_hdf('../../../Data/test_table.hdf', 'test')

def test_hdf_table_write_compress(df):
    df.to_hdf('../../../Data/test_table_compress.hdf', 'test', mode='w',
              complib='blosc', format='table')

def test_hdf_table_read_compress():
    pd.read_hdf('../../../Data/test_table_compress.hdf', 'test')

def test_csv_write(df):
    df.to_csv('../../../Data/test.csv', mode='w')

def test_csv_read():
    pd.read_csv('../../../Data/test.csv', index_col=0)

def test_feather_write(df):
    df.to_feather('../../../Data/test.feather')

def test_feather_read():
    pd.read_feather('../../../Data/test.feather')

def test_pickle_write(df):
    df.to_pickle('../../../Data/test.pkl')

def test_pickle_read():
    pd.read_pickle('../../../Data/test.pkl')

def test_pickle_write_compress(df):
    df.to_pickle('../../../Data/test.pkl.compress', compression='xz')

def test_pickle_read_compress():
    pd.read_pickle('../../../Data/test.pkl.compress', compression='xz')

def test_parquet_write(df):
    df.to_parquet('../../../Data/test.parquet')

def test_parquet_read():
    pd.read_parquet('../../../Data/test.parquet')

# Write

#data = pd.read_fwf('../../../Data/TCs_air5.txt')
#df = pd.DataFrame(data)

#start_time = time.time()
#test_hdf_fixed_write(df)
#print("test_hdf_fixed_write --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_fixed_write(df)
#print("test_hdf_fixed_write --- %s seconds ---" % (time.time() - start_time))
#
##start_time = time.time()
##test_sql_write(df)
##print("test_sql_write --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_fixed_write(df)
#print("test_hdf_fixed_write --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_fixed_write_compress(df)
#print("test_hdf_fixed_write_compress --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_table_write(df)
#print("test_hdf_table_write --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_table_write_compress(df)
#print("test_hdf_table_write_compress --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_csv_write(df)
#print("test_csv_write --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_feather_write(df)
#print("test_feather_write --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_pickle_write(df)
#print("test_pickle_write --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_pickle_write_compress(df)
#print("test_pickle_write_compress --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_parquet_write(df)
#print("test_parquet_write --- %s seconds ---" % (time.time() - start_time))
#
# Read
#
#start_time = time.time()
#test_sql_read()
#print("test_sql_read --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_fixed_read()
#print("test_hdf_fixed_read --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_fixed_read_compress()
#print("test_hdf_fixed_read_compress --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_table_read()
#print("test_hdf_table_read --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_hdf_table_read_compress()
#print("test_hdf_table_read_compress --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_csv_read()
#print("test_csv_read --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_feather_read()
#print("test_feather_read --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_pickle_read()
#print("test_pickle_read --- %s seconds ---" % (time.time() - start_time))
#
#start_time = time.time()
#test_pickle_read_compress()
#print("test_pickle_read_compress --- %s seconds ---" % (time.time() - start_time))

## The data is then split into training and test data
#x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.75, test_size=0.25, random_state=69)
#
#sc_x = StandardScaler()
#sc_y = StandardScaler()
#
#sc_x.fit(x_train)
#x_train = sc_x.fit_transform(x_train)
#x_test  = sc_x.fit_transform(x_test)
#
#sc_y.fit(y_train)
#y_train = sc_y.transform(y_train)
#y_test  = sc_y.transform(y_test)
#
#dump(sc_x, open('../scaler/scaler_x_MD.pkl', 'wb'))
#dump(sc_y, open('../scaler/scaler_y_MD.pkl', 'wb'))
#
#print('Training Features Shape:', x_train.shape)
#print('Training Labels Shape:', y_train.shape)
#print('Testing Features Shape:', x_test.shape)
#print('Testing Labels Shape:', y_test.shape)
#
#regr = DecisionTreeRegressor(criterion='mse',
#                             splitter='best',
#                             max_features='auto',
#                             random_state=69)
#
#regr = MultiOutputRegressor(estimator=regr)
#
#t0 = time.time()
#with parallel_backend("dask"):
#    regr.fit(x_train, y_train)
#regr_fit = time.time() - t0
#print("Complexity and bandwidth selected and model fitted in %.6f s" % regr_fit)
#
#t0 = time.time()
#y_regr = regr.predict(x_test)
#regr_predict = time.time() - t0
#print("Prediction for %d inputs in %.6f s" % (x_test.shape[0], regr_predict))
#
#x_test_dim = sc_x.inverse_transform(x_test)
#y_test_dim = sc_y.inverse_transform(y_test)
#y_regr_dim = sc_y.inverse_transform(y_regr)
#
#plt.scatter(x_test_dim[:,0], y_test_dim[:,0], s=5, c='k', marker='o', label='KAPPA')
#plt.scatter(x_test_dim[:,0], y_regr_dim[:,0], s=5, c='r', marker='d', label='k-Nearest Neighbour')
#plt.scatter(x_test_dim[:,0], y_test_dim[:,1], s=5, c='k', marker='o', label='KAPPA')
#plt.scatter(x_test_dim[:,0], y_regr_dim[:,1], s=5, c='r', marker='d', label='k-Nearest Neighbour')
##plt.scatter(x_test_dim[:,0], y_test_dim[:,2], s=5, c='k', marker='o', label='KAPPA')
##plt.scatter(x_test_dim[:,0], y_regr_dim[:,2], s=5, c='r', marker='d', label='k-Nearest Neighbour')
##plt.title('Shear viscosity regression with kNN')
##plt.ylabel(r'$\eta$ [Pa·s]')
#plt.ylabel(' ')
#plt.xlabel('T [K] ')
#plt.legend()
#plt.tight_layout()
#plt.savefig("../pdf/regression_MD.pdf", dpi=150, crop='false')
#plt.show()
#
## save the model to disk
#dump(regr, '../model/model_MD.sav')

/home/lk/.local/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46305 instead
  http_address["port"], self.http_server.port
